In [ ]:
import os
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array
import tkinter as tk
from tkinter import messagebox, simpledialog
import csv
from datetime import datetime
import smtplib
from email.mime.text import MIMEText
import dlib

# === PATHS ===
PROTOTXT_PATH = "/Users/sagarpetwal/Downloads/PROJECT/deploy.prototxt" 
CAFFEMODEL_PATH = "/Users/sagarpetwal/Downloads/PROJECT/res10_300x300_ssd_iter_140000.caffemodel"
MODEL_PATH = "/Users/sagarpetwal/Downloads/PROJECT/cnn_face_recog.keras"
DATASET_PATH = "/Users/sagarpetwal/Downloads/PROJECT/dataset"
ATTENDANCE_FILE = "/Users/sagarpetwal/Downloads/PROJECT/attendance.csv"
SHAPE_PREDICTOR_PATH = "/Users/sagarpetwal/Downloads/PROJECT/shape_predictor_68_face_landmarks.dat"

# === EMAIL CONFIG ===
SENDER_EMAIL = "SENDEREMAIL@GMAIL.COM" # INSERT SENDER EMAIL WITH 2FA ENBLED
SENDER_PASSWORD = "PASSWORD" #PASSWORD
RECIPIENT_EMAIL = "RECIPIENTMAIL@GMAIL.COM" # INSERT RECIPIENT MAIL

# === MODEL & DNN ===
net = cv2.dnn.readNetFromCaffe(PROTOTXT_PATH, CAFFEMODEL_PATH)
model = load_model(MODEL_PATH)
class_labels = sorted([d for d in os.listdir(DATASET_PATH) if os.path.isdir(os.path.join(DATASET_PATH, d)) and not d.startswith('.')])

# === EMAIL FUNCTION ===
def send_email_report(attendance_list):
    subject = "Daily Attendance Report"
    body = "Attendance marked for the day:\n\n" + "\n".join(attendance_list)
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = SENDER_EMAIL
    msg['To'] = RECIPIENT_EMAIL
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(SENDER_EMAIL, SENDER_PASSWORD)
        server.sendmail(SENDER_EMAIL, RECIPIENT_EMAIL, msg.as_string())
        server.quit()
        print(" Email sent successfully.")
    except Exception as e:
        print(f" Failed to send email: {e}")

# === MARK ATTENDANCE ===
def mark_attendance(name):
    now = datetime.now()
    time_str = now.strftime("%H:%M:%S")
    date_str = now.strftime("%Y-%m-%d")
    if not os.path.isfile(ATTENDANCE_FILE):
        with open(ATTENDANCE_FILE, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Name", "Date", "Time"])
    with open(ATTENDANCE_FILE, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([name, date_str, time_str])

# === REGISTER STUDENT ===
def register_student():
    from keras.models import Sequential
    from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
    from keras.utils import to_categorical
    from sklearn.preprocessing import LabelEncoder

    def train_model(dataset_path, model_save_path):
        images, labels = [], []
        for person_name in os.listdir(dataset_path):
            person_path = os.path.join(dataset_path, person_name)
            if not os.path.isdir(person_path) or person_name.startswith('.'):
                continue
            for image_name in os.listdir(person_path):
                image_path = os.path.join(person_path, image_name)
                img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue
                img = cv2.resize(img, (100, 100))
                images.append(img)
                labels.append(person_name)

        le = LabelEncoder()
        labels_num = le.fit_transform(labels)
        X = np.array(images).reshape(-1, 100, 100, 1).astype("float32") / 255.0
        y = to_categorical(labels_num)

        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(100, activation='relu'),
            Dense(len(np.unique(labels)), activation='softmax')
        ])
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(X, y, epochs=10, verbose=1)
        model.save(model_save_path)
        return model, le

    name = simpledialog.askstring("Register Student", "Enter student name:")
    if not name:
        return

    path = os.path.join(DATASET_PATH, name)
    os.makedirs(path, exist_ok=True)
    cap = cv2.VideoCapture(0)
    count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        h, w = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.7:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x1, y1, x2, y2) = box.astype("int")
                face = frame[y1:y2, x1:x2]
                if face.size == 0:
                    continue
                face = cv2.resize(face, (100, 100))
                cv2.imwrite(os.path.join(path, f"{count}.jpg"), face)
                count += 1
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.imshow("Register - Press Q to exit", frame)
        if cv2.waitKey(1) & 0xFF == ord('q') or count >= 50:
            break
    cap.release()
    cv2.destroyAllWindows()
    messagebox.showinfo("Success", f"Captured {count} images for {name}.")

    # Retrain model
    global model, class_labels
    model, label_encoder = train_model(DATASET_PATH, MODEL_PATH)
    class_labels = list(label_encoder.classes_)

# === START ATTENDANCE ===
def start_attendance():
    cap = cv2.VideoCapture(0)
    attendance_today = set()
    predictor = dlib.shape_predictor(SHAPE_PREDICTOR_PATH)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        h, w = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.7:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x1, y1, x2, y2) = box.astype("int")
                face = frame[y1:y2, x1:x2]
                if face.size == 0:
                    continue
                face_gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                face_gray = cv2.resize(face_gray, (100, 100))
                face_gray = face_gray.astype("float32") / 255.0
                face_gray = np.expand_dims(face_gray, axis=-1)
                face_input = np.expand_dims(face_gray, axis=0)
                pred = model.predict(face_input, verbose=0)[0]
                label_idx = np.argmax(pred)
                name = class_labels[label_idx]

                if name not in attendance_today:
                    mark_attendance(name)
                    attendance_today.add(name)

                label = f"{name}: {pred[label_idx]*100:.2f}%"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        cv2.imshow("Attendance - Press Q to exit", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

    if attendance_today:
        send_email_report(list(attendance_today))
        messagebox.showinfo("Report", "Email report sent!")

# === GUI ===
root = tk.Tk()
root.title("Face Recognition Attendance System")
root.geometry("400x200")

tk.Button(root, text="Register Student", command=register_student, font=('Arial', 14)).pack(pady=20)
tk.Button(root, text="Start Attendance", command=start_attendance, font=('Arial', 14)).pack(pady=20)

root.mainloop()
